In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [5]:
import pandas as pd
adm_data=pd.read_excel("/Users/bhaskaravanacharla/Downloads/Documents/Machine_learning/admissionnn/Fall 2025 3.3.25.xlsx")

In [6]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12094 entries, 0 to 12093
Data columns (total 35 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12094 non-null  int64         
 1   Application Entry Term               12094 non-null  object        
 2   Application Major                    12094 non-null  object        
 3   Application College                  12094 non-null  object        
 4   Application Housing                  11572 non-null  object        
 5   Application Enroll Status            12094 non-null  object        
 6   Application Submitted Date           12094 non-null  datetime64[ns]
 7   Application Completed Date           12094 non-null  datetime64[ns]
 8   Admit Confirmed Date                 12094 non-null  datetime64[ns]
 9   Admit Received/Opened Date           6778 non-null   datetime64[ns]
 10  Admit Open

In [7]:
adm_data['admitted']=adm_data['Deposit Confirmed Date'].apply(lambda x:1 if pd.notna(x) and x != '' else 0)
adm_data['admitted'].value_counts()
adm_data.drop(columns=['Deposit Confirmed Date'], inplace=True)

In [8]:
adm_data['admitted'].value_counts()

admitted
0    11800
1      294
Name: count, dtype: int64

In [9]:
## Keep only U.S.-based rows
#to make sure the index is reset and start from 0 to n-1 else it will repeat the old index 
adm_data = adm_data[adm_data['High School Country'] == "United States"].reset_index(drop=True)
adm_data = adm_data[adm_data['High School Country'] == "United States"].dropna(axis=0, how='all')

In [10]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12051 entries, 0 to 12050
Data columns (total 35 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12051 non-null  int64         
 1   Application Entry Term               12051 non-null  object        
 2   Application Major                    12051 non-null  object        
 3   Application College                  12051 non-null  object        
 4   Application Housing                  11545 non-null  object        
 5   Application Enroll Status            12051 non-null  object        
 6   Application Submitted Date           12051 non-null  datetime64[ns]
 7   Application Completed Date           12051 non-null  datetime64[ns]
 8   Admit Confirmed Date                 12051 non-null  datetime64[ns]
 9   Admit Received/Opened Date           6753 non-null   datetime64[ns]
 10  Admit Open

In [11]:
#region mapping for the regions of the 
# Dictionary to map full state names to abbreviations
state_mapping = {
    "Alaska": "AK",
    "Alabama": "AL",
    "Armed Forces Pacific": "AP",
    "Arkansas": "AR",
    "Arizona": "AZ",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "District of Columbia": "DC",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Guam": "GU",
    "Hawaii": "HI",
    "Iowa": "IA",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Massachusetts": "MA",
    "Maryland": "MD",
    "Maine": "ME",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Missouri": "MO",
    "Mississippi": "MS",
    "Montana": "MT",
    "North Carolina": "NC",
    "Nebraska": "NE",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "Nevada": "NV",
    "New York": "NY",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Puerto Rico": "PR",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Virginia": "VA",
    "Virgin Islands": "VI",
    "Vermont": "VT",
    "Washington": "WA",
    "Wisconsin": "WI",
    "West Virginia": "WV"
}

# Standardize the 'Region' column by replacing full state names with abbreviations
#adm_data['Address 1 Region'] = adm_data['Address 1 Region '].replace(state_mapping)

# Remove rows where 'Region' is still not a valid U.S. state abbreviation
valid_states = set(state_mapping.values())

#Drop non-U.S. rows (anything that wasn't mapped) and make sure to reset the index 
adm_data = adm_data[adm_data['Address 1 Region'].isin(valid_states)].reset_index(drop=True)


In [12]:
#converting states to regions
states_to_regions = {
    # West
    'WA': 'West', 'OR': 'West', 'CA': 'West', 'NV': 'West',
    'ID': 'West', 'MT': 'West', 'WY': 'West', 'UT': 'West',
    'CO': 'West', 'AK': 'West', 'HI': 'West',
    
    # Northeast
    'ME': 'Northeast', 'VT': 'Northeast', 'NH': 'Northeast', 
    'MA': 'Northeast', 'RI': 'Northeast', 'CT': 'Northeast',
    'NY': 'Northeast', 'NJ': 'Northeast', 'PA': 'Northeast',
    
    # Midwest
    'ND': 'Midwest', 'SD': 'Midwest', 'NE': 'Midwest', 'KS': 'Midwest',
    'MN': 'Midwest', 'IA': 'Midwest', 'MO': 'Midwest', 'WI': 'Midwest',
    'IL': 'Midwest', 'MI': 'Midwest', 'IN': 'Midwest', 'OH': 'Midwest',
    
    # South
    'WV': 'South', 'DC': 'South', 'MD': 'South', 'VA': 'South',
    'KY': 'South', 'TN': 'South', 'NC': 'South', 'MS': 'South',
    'AR': 'South', 'LA': 'South', 'AL': 'South', 'GA': 'South',
    'SC': 'South', 'FL': 'South', 'DE': 'South',
    
    # Southwest
    'AZ': 'Southwest', 'NM': 'Southwest', 'OK': 'Southwest', 'TX': 'Southwest',
    
    # Unclassified / Territories
    'PR': 'Territory',  # Puerto Rico
    'GU': 'Territory',  # Guam
    'VI': 'Territory',  # Virgin Islands
    'AP': 'Military',   # Armed Forces Pacific
}

adm_data['Address 1 Region']=adm_data['Address 1 Region'].map(states_to_regions)

In [13]:
#one hot encode them 
encoder= OneHotEncoder(sparse_output=False, dtype=int)
encoded_array = encoder.fit_transform(adm_data[['Address 1 Region']])
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['Address 1 Region']))
adm_data = pd.concat([adm_data, encoded_df], axis=1)

In [14]:
adm_data['admitted'].value_counts()

admitted
0    11754
1      294
Name: count, dtype: int64

In [15]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12048 entries, 0 to 12047
Data columns (total 41 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12048 non-null  int64         
 1   Application Entry Term               12048 non-null  object        
 2   Application Major                    12048 non-null  object        
 3   Application College                  12048 non-null  object        
 4   Application Housing                  11543 non-null  object        
 5   Application Enroll Status            12048 non-null  object        
 6   Application Submitted Date           12048 non-null  datetime64[ns]
 7   Application Completed Date           12048 non-null  datetime64[ns]
 8   Admit Confirmed Date                 12048 non-null  datetime64[ns]
 9   Admit Received/Opened Date           6750 non-null   datetime64[ns]
 10  Admit Open

In [16]:
adm_data.head()

,Application Reference ID,Application Entry Term,Application Major,Application College,Application Housing,Application Enroll Status,Application Submitted Date,Application Completed Date,Admit Confirmed Date,Admit Received/Opened Date,...,Emails Opened,Was Inquiry,Athlete,admitted,Address 1 Region_Midwest,Address 1 Region_Northeast,Address 1 Region_South,Address 1 Region_Southwest,Address 1 Region_Territory,Address 1 Region_West
0,875213089,Fall 2025,Legal Studies - Paralegal Studies,CAS,Residential,Full Time,2024-11-06,2024-11-25,2024-11-27 11:00:21,NaT,...,13,1,0,0,0,1,0,0,0,0
1,662645710,Fall 2025,Health Sciences - General,SHS,Residential,Full Time,2024-11-23,2024-12-11,2024-12-12 16:45:16,NaT,...,0,0,0,0,0,1,0,0,0,0
2,865216777,Fall 2025,Exercise Science,SHS,Residential,Full Time,2024-11-01,2024-11-12,2024-11-13 09:57:40,NaT,...,15,1,0,0,0,1,0,0,0,0
3,845896813,Fall 2025,Civil Engineering,TCOE,Residential,Full Time,2024-11-04,2024-11-26,2024-12-03 09:23:14,2024-12-03 12:37:10,...,12,1,0,0,0,1,0,0,0,0
4,83776898,Fall 2025,Undeclared,CAS,Residential,Full Time,2024-12-06,2024-12-10,2024-12-10 16:01:31,NaT,...,0,0,0,0,0,1,0,0,0,0


In [17]:
adm_data.dtypes

Application Reference ID                        int64
Application Entry Term                         object
Application Major                              object
Application College                            object
Application Housing                            object
Application Enroll Status                      object
Application Submitted Date             datetime64[ns]
Application Completed Date             datetime64[ns]
Admit Confirmed Date                   datetime64[ns]
Admit Received/Opened Date             datetime64[ns]
Admit Open Span                               float64
Scholarship_Awarded                             int64
Application CGPA                               object
Person Sex                                     object
FAFSA Filed                                     int64
High School Name                               object
High School Code                              float64
High School City                               object
High School Region          

In [18]:
#2 convert application to two categories using one hot encoding or label encoder
encoder= OneHotEncoder(sparse_output=False, dtype=int)
encoded_array = encoder.fit_transform(adm_data[['Application Housing']])
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['Application Housing']))
adm_data = pd.concat([adm_data, encoded_df], axis=1)

In [19]:
adm_data.drop(columns=['Application Housing'], inplace=True)
#drop the column else we get a new dataframe with the columns 

In [20]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12048 entries, 0 to 12047
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12048 non-null  int64         
 1   Application Entry Term               12048 non-null  object        
 2   Application Major                    12048 non-null  object        
 3   Application College                  12048 non-null  object        
 4   Application Enroll Status            12048 non-null  object        
 5   Application Submitted Date           12048 non-null  datetime64[ns]
 6   Application Completed Date           12048 non-null  datetime64[ns]
 7   Admit Confirmed Date                 12048 non-null  datetime64[ns]
 8   Admit Received/Opened Date           6750 non-null   datetime64[ns]
 9   Admit Open Span                      6750 non-null   float64       
 10  Scholarshi

In [21]:
#3 convert application enrolled to two categories using one hot encoding or label encoder
encoder= OneHotEncoder(sparse_output=False, dtype=int)
encoded_array = encoder.fit_transform(adm_data[['Application Enroll Status']])
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['Application Enroll Status']))
adm_data = pd.concat([adm_data, encoded_df], axis=1)
adm_data.drop(columns=['Application Enroll Status'], inplace=True)

In [22]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12048 entries, 0 to 12047
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12048 non-null  int64         
 1   Application Entry Term               12048 non-null  object        
 2   Application Major                    12048 non-null  object        
 3   Application College                  12048 non-null  object        
 4   Application Submitted Date           12048 non-null  datetime64[ns]
 5   Application Completed Date           12048 non-null  datetime64[ns]
 6   Admit Confirmed Date                 12048 non-null  datetime64[ns]
 7   Admit Received/Opened Date           6750 non-null   datetime64[ns]
 8   Admit Open Span                      6750 non-null   float64       
 9   Scholarship_Awarded                  12048 non-null  int64         
 10  Applicatio

In [23]:
#person sex to two categories 
encoder= OneHotEncoder(sparse_output=False, dtype=int)
encoded_array = encoder.fit_transform(adm_data[['Person Sex']])
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['Person Sex']))
adm_data = pd.concat([adm_data, encoded_df], axis=1)
adm_data.drop(columns=['Person Sex'], inplace=True)

In [24]:
adm_data.rename(columns={'Person Sex_nan': 'Person Sex_Unknown'}, inplace=True)


In [25]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12048 entries, 0 to 12047
Data columns (total 45 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12048 non-null  int64         
 1   Application Entry Term               12048 non-null  object        
 2   Application Major                    12048 non-null  object        
 3   Application College                  12048 non-null  object        
 4   Application Submitted Date           12048 non-null  datetime64[ns]
 5   Application Completed Date           12048 non-null  datetime64[ns]
 6   Admit Confirmed Date                 12048 non-null  datetime64[ns]
 7   Admit Received/Opened Date           6750 non-null   datetime64[ns]
 8   Admit Open Span                      6750 non-null   float64       
 9   Scholarship_Awarded                  12048 non-null  int64         
 10  Applicatio

In [26]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12048 entries, 0 to 12047
Data columns (total 45 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12048 non-null  int64         
 1   Application Entry Term               12048 non-null  object        
 2   Application Major                    12048 non-null  object        
 3   Application College                  12048 non-null  object        
 4   Application Submitted Date           12048 non-null  datetime64[ns]
 5   Application Completed Date           12048 non-null  datetime64[ns]
 6   Admit Confirmed Date                 12048 non-null  datetime64[ns]
 7   Admit Received/Opened Date           6750 non-null   datetime64[ns]
 8   Admit Open Span                      6750 non-null   float64       
 9   Scholarship_Awarded                  12048 non-null  int64         
 10  Applicatio

In [27]:
adm_data.drop('High School Name',axis=1,inplace=True)

In [28]:
adm_data.drop('High School City',axis=1,inplace=True)

In [29]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12048 entries, 0 to 12047
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12048 non-null  int64         
 1   Application Entry Term               12048 non-null  object        
 2   Application Major                    12048 non-null  object        
 3   Application College                  12048 non-null  object        
 4   Application Submitted Date           12048 non-null  datetime64[ns]
 5   Application Completed Date           12048 non-null  datetime64[ns]
 6   Admit Confirmed Date                 12048 non-null  datetime64[ns]
 7   Admit Received/Opened Date           6750 non-null   datetime64[ns]
 8   Admit Open Span                      6750 non-null   float64       
 9   Scholarship_Awarded                  12048 non-null  int64         
 10  Applicatio

In [30]:
#fix the null areas of high school regions and check with the field admitted and drop them if they are not admitted 
#adm_data[adm_data['High School Region'].isnull()]['admitted']
adm_data = adm_data.drop(adm_data[(adm_data['High School Region'].isnull()) & (adm_data['admitted'] == 0)].index)


In [31]:
#adm_data['High School Region'].isnull().sum()
#now fix the High school region nan value with the school name and map it with the respective state of school
#adm_data[adm_data['High School Region'].isnull()]
adm_data.loc[(adm_data['High School Region'].isnull()), 'High School Region'] = 'CT'


In [32]:
#region mapping for the regions of the 
# Dictionary to map full state names to abbreviations
state_mapping = {
    "Alaska": "AK",
    "Alabama": "AL",
    "Armed Forces Pacific": "AP",
    "Arkansas": "AR",
    "Arizona": "AZ",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "District of Columbia": "DC",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Guam": "GU",
    "Hawaii": "HI",
    "Iowa": "IA",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Massachusetts": "MA",
    "Maryland": "MD",
    "Maine": "ME",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Missouri": "MO",
    "Mississippi": "MS",
    "Montana": "MT",
    "North Carolina": "NC",
    "Nebraska": "NE",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "Nevada": "NV",
    "New York": "NY",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Puerto Rico": "PR",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Virginia": "VA",
    "Virgin Islands": "VI",
    "Vermont": "VT",
    "Washington": "WA",
    "Wisconsin": "WI",
    "West Virginia": "WV"
}


states_to_regions = {
    # West
    'WA': 'West', 'OR': 'West', 'CA': 'West', 'NV': 'West',
    'ID': 'West', 'MT': 'West', 'WY': 'West', 'UT': 'West',
    'CO': 'West', 'AK': 'West', 'HI': 'West',
    
    # Northeast
    'ME': 'Northeast', 'VT': 'Northeast', 'NH': 'Northeast', 
    'MA': 'Northeast', 'RI': 'Northeast', 'CT': 'Northeast',
    'NY': 'Northeast', 'NJ': 'Northeast', 'PA': 'Northeast',
    
    # Midwest
    'ND': 'Midwest', 'SD': 'Midwest', 'NE': 'Midwest', 'KS': 'Midwest',
    'MN': 'Midwest', 'IA': 'Midwest', 'MO': 'Midwest', 'WI': 'Midwest',
    'IL': 'Midwest', 'MI': 'Midwest', 'IN': 'Midwest', 'OH': 'Midwest',
    
    # South
    'WV': 'South', 'DC': 'South', 'MD': 'South', 'VA': 'South',
    'KY': 'South', 'TN': 'South', 'NC': 'South', 'MS': 'South',
    'AR': 'South', 'LA': 'South', 'AL': 'South', 'GA': 'South',
    'SC': 'South', 'FL': 'South', 'DE': 'South',
    
    # Southwest
    'AZ': 'Southwest', 'NM': 'Southwest', 'OK': 'Southwest', 'TX': 'Southwest',
    
    # Unclassified / Territories
    'PR': 'Territory',  # Puerto Rico
    'GU': 'Territory',  # Guam
    'VI': 'Territory',  # Virgin Islands
    'AP': 'Military',   # Armed Forces Pacific
}


# Standardize the 'Region' column by replacing full state names with abbreviations
adm_data['High School Region'] = adm_data['High School Region'].replace(state_mapping)
# Remove rows where 'Region' is still not a valid U.S. state abbreviation
valid_states = set(state_mapping.values())
#Drop non-U.S. rows (anything that wasn't mapped) and make sure to reset the index 
adm_data = adm_data[adm_data['High School Region'].isin(valid_states)].reset_index(drop=True)

adm_data['High School Region']=adm_data['High School Region'].map(states_to_regions)



In [33]:
#one hot encode them 
encoder= OneHotEncoder(sparse_output=False, dtype=int)
encoded_array = encoder.fit_transform(adm_data[['High School Region']])
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['High School Region']))
adm_data = pd.concat([adm_data, encoded_df], axis=1)

In [34]:
adm_data.drop('Address 1 Region',axis=1,inplace=True)

In [35]:
adm_data.drop('Application Entry Term',axis=1,inplace=True)

In [36]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12045 entries, 0 to 12044
Data columns (total 47 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12045 non-null  int64         
 1   Application Major                    12045 non-null  object        
 2   Application College                  12045 non-null  object        
 3   Application Submitted Date           12045 non-null  datetime64[ns]
 4   Application Completed Date           12045 non-null  datetime64[ns]
 5   Admit Confirmed Date                 12045 non-null  datetime64[ns]
 6   Admit Received/Opened Date           6747 non-null   datetime64[ns]
 7   Admit Open Span                      6747 non-null   float64       
 8   Scholarship_Awarded                  12045 non-null  int64         
 9   Application CGPA                     12045 non-null  object        
 10  FAFSA File

In [37]:
adm_data['Application Consider Test Scores']=adm_data['Application Consider Test Scores'].apply(lambda x: 0 if x == "Do not consider my SAT and/or ACT results" or pd.isna(x) else 1)


In [38]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12045 entries, 0 to 12044
Data columns (total 47 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12045 non-null  int64         
 1   Application Major                    12045 non-null  object        
 2   Application College                  12045 non-null  object        
 3   Application Submitted Date           12045 non-null  datetime64[ns]
 4   Application Completed Date           12045 non-null  datetime64[ns]
 5   Admit Confirmed Date                 12045 non-null  datetime64[ns]
 6   Admit Received/Opened Date           6747 non-null   datetime64[ns]
 7   Admit Open Span                      6747 non-null   float64       
 8   Scholarship_Awarded                  12045 non-null  int64         
 9   Application CGPA                     12045 non-null  object        
 10  FAFSA File

In [39]:
adm_data['Application CGPA'] = pd.to_numeric(adm_data['Application CGPA'], errors='coerce')

In [40]:
adm_data['Application CGPA']=adm_data['Application CGPA'].apply(lambda x :4 if x>=4 else x)

In [41]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12045 entries, 0 to 12044
Data columns (total 47 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12045 non-null  int64         
 1   Application Major                    12045 non-null  object        
 2   Application College                  12045 non-null  object        
 3   Application Submitted Date           12045 non-null  datetime64[ns]
 4   Application Completed Date           12045 non-null  datetime64[ns]
 5   Admit Confirmed Date                 12045 non-null  datetime64[ns]
 6   Admit Received/Opened Date           6747 non-null   datetime64[ns]
 7   Admit Open Span                      6747 non-null   float64       
 8   Scholarship_Awarded                  12045 non-null  int64         
 9   Application CGPA                     12043 non-null  float64       
 10  FAFSA File

In [42]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12045 entries, 0 to 12044
Data columns (total 47 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12045 non-null  int64         
 1   Application Major                    12045 non-null  object        
 2   Application College                  12045 non-null  object        
 3   Application Submitted Date           12045 non-null  datetime64[ns]
 4   Application Completed Date           12045 non-null  datetime64[ns]
 5   Admit Confirmed Date                 12045 non-null  datetime64[ns]
 6   Admit Received/Opened Date           6747 non-null   datetime64[ns]
 7   Admit Open Span                      6747 non-null   float64       
 8   Scholarship_Awarded                  12045 non-null  int64         
 9   Application CGPA                     12043 non-null  float64       
 10  FAFSA File

In [43]:
adm_data['Application Span'] = (adm_data['Application Completed Date'] - adm_data['Application Submitted Date']).dt.days

In [44]:
# Drop the 'Application Submitted Date' and 'Application Completed Date' columns
adm_data.drop(['Application Submitted Date', 'Application Completed Date'], axis=1,inplace=True)


In [45]:
#adm_data.drop('Application Housing_nan',axis=1,inplace=True)

In [46]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12045 entries, 0 to 12044
Data columns (total 46 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12045 non-null  int64         
 1   Application Major                    12045 non-null  object        
 2   Application College                  12045 non-null  object        
 3   Admit Confirmed Date                 12045 non-null  datetime64[ns]
 4   Admit Received/Opened Date           6747 non-null   datetime64[ns]
 5   Admit Open Span                      6747 non-null   float64       
 6   Scholarship_Awarded                  12045 non-null  int64         
 7   Application CGPA                     12043 non-null  float64       
 8   FAFSA Filed                          12045 non-null  int64         
 9   High School Code                     12028 non-null  float64       
 10  High Schoo

In [47]:
adm_data['High School Region'].value_counts()

High School Region
Northeast    10828
South          592
West           264
Midwest        170
Southwest      118
Territory       73
Name: count, dtype: int64

In [48]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12045 entries, 0 to 12044
Data columns (total 46 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12045 non-null  int64         
 1   Application Major                    12045 non-null  object        
 2   Application College                  12045 non-null  object        
 3   Admit Confirmed Date                 12045 non-null  datetime64[ns]
 4   Admit Received/Opened Date           6747 non-null   datetime64[ns]
 5   Admit Open Span                      6747 non-null   float64       
 6   Scholarship_Awarded                  12045 non-null  int64         
 7   Application CGPA                     12043 non-null  float64       
 8   FAFSA Filed                          12045 non-null  int64         
 9   High School Code                     12028 non-null  float64       
 10  High Schoo

In [49]:
adm_data.drop('High School Region',axis=1,inplace=True)

In [50]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12045 entries, 0 to 12044
Data columns (total 45 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12045 non-null  int64         
 1   Application Major                    12045 non-null  object        
 2   Application College                  12045 non-null  object        
 3   Admit Confirmed Date                 12045 non-null  datetime64[ns]
 4   Admit Received/Opened Date           6747 non-null   datetime64[ns]
 5   Admit Open Span                      6747 non-null   float64       
 6   Scholarship_Awarded                  12045 non-null  int64         
 7   Application CGPA                     12043 non-null  float64       
 8   FAFSA Filed                          12045 non-null  int64         
 9   High School Code                     12028 non-null  float64       
 10  High Schoo

In [51]:
adm_data['Admit Received/Opened Date']=adm_data['Admit Received/Opened Date'].fillna(adm_data['Admit Confirmed Date'])

In [52]:
adm_data['Admission Span'] = (adm_data['Admit Received/Opened Date']-adm_data['Admit Confirmed Date']).dt.total_seconds()
adm_data['Admission Span'] = adm_data['Admission Span']/ 86400

In [53]:
# Drop the 'Application Submitted Date' and 'Application Completed Date' columns
adm_data.drop(['Admit Received/Opened Date', 'Admit Confirmed Date'], axis=1,inplace=True)


In [54]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12045 entries, 0 to 12044
Data columns (total 44 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Application Reference ID             12045 non-null  int64         
 1   Application Major                    12045 non-null  object        
 2   Application College                  12045 non-null  object        
 3   Admit Open Span                      6747 non-null   float64       
 4   Scholarship_Awarded                  12045 non-null  int64         
 5   Application CGPA                     12043 non-null  float64       
 6   FAFSA Filed                          12045 non-null  int64         
 7   High School Code                     12028 non-null  float64       
 8   High School Country                  12045 non-null  object        
 9   Financial Aid Appeal                 12045 non-null  int64         
 10  Accepted S

In [55]:
adm_data.drop('High School Country',axis=1,inplace=True)

In [56]:
adm_data['Waitlist Confirmed Date'] = adm_data['Waitlist Confirmed Date'].notna().astype(int)


In [57]:
adm_data['Waitlist Confirmed Date'].value_counts()

Waitlist Confirmed Date
0    12019
1       26
Name: count, dtype: int64

In [58]:
adm_data.drop('Application Reference ID',axis=1,inplace=True)

In [59]:
adm_data.head()

,Application Major,Application College,Admit Open Span,Scholarship_Awarded,Application CGPA,FAFSA Filed,High School Code,Financial Aid Appeal,Accepted Student Day Event Attended,Campus Visits - Person,...,Person Sex_M,Person Sex_Unknown,High School Region_Midwest,High School Region_Northeast,High School Region_South,High School Region_Southwest,High School Region_Territory,High School Region_West,Application Span,Admission Span
0,Legal Studies - Paralegal Studies,CAS,NaN,1,3.7,0,311120.0,0,0,0,...,0,0,0,1,0,0,0,0,19,0.000000
1,Health Sciences - General,SHS,NaN,1,4.0,0,201100.0,0,0,0,...,0,0,0,1,0,0,0,0,18,0.000000
2,Exercise Science,SHS,NaN,1,3.8,1,70905.0,0,0,0,...,1,0,0,1,0,0,0,0,11,0.000000
3,Civil Engineering,TCOE,0.0,1,3.5,1,311073.0,0,0,0,...,0,0,0,1,0,0,0,0,22,0.134676
4,Undeclared,CAS,NaN,1,3.2,1,70750.0,0,0,0,...,1,0,0,1,0,0,0,0,4,0.000000


In [60]:
label_encoder = LabelEncoder()
adm_data['Application Major'] = label_encoder.fit_transform(adm_data['Application Major'])

In [61]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12045 entries, 0 to 12044
Data columns (total 42 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Application Major                    12045 non-null  int64  
 1   Application College                  12045 non-null  object 
 2   Admit Open Span                      6747 non-null   float64
 3   Scholarship_Awarded                  12045 non-null  int64  
 4   Application CGPA                     12043 non-null  float64
 5   FAFSA Filed                          12045 non-null  int64  
 6   High School Code                     12028 non-null  float64
 7   Financial Aid Appeal                 12045 non-null  int64  
 8   Accepted Student Day Event Attended  12045 non-null  int64  
 9   Campus Visits - Person               12045 non-null  int64  
 10  Campus Visits - App                  12045 non-null  int64  
 11  Logins Before Admit         

In [62]:
adm_data[adm_data['Application CGPA'].isnull()]


,Application Major,Application College,Admit Open Span,Scholarship_Awarded,Application CGPA,FAFSA Filed,High School Code,Financial Aid Appeal,Accepted Student Day Event Attended,Campus Visits - Person,...,Person Sex_M,Person Sex_Unknown,High School Region_Midwest,High School Region_Northeast,High School Region_South,High School Region_Southwest,High School Region_Territory,High School Region_West,Application Span,Admission Span
2459,58,HCLC,0.0,1,NaN,1,70430.0,0,0,0,...,0,0,0,1,0,0,0,0,9,0.102106
10213,2,CAS,1.0,1,NaN,1,220063.0,0,0,0,...,0,0,0,1,0,0,0,0,21,1.096921


In [63]:
adm_data = adm_data.dropna(subset=['Application CGPA'])

In [64]:
adm_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 12043 entries, 0 to 12044
Data columns (total 42 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Application Major                    12043 non-null  int64  
 1   Application College                  12043 non-null  object 
 2   Admit Open Span                      6745 non-null   float64
 3   Scholarship_Awarded                  12043 non-null  int64  
 4   Application CGPA                     12043 non-null  float64
 5   FAFSA Filed                          12043 non-null  int64  
 6   High School Code                     12026 non-null  float64
 7   Financial Aid Appeal                 12043 non-null  int64  
 8   Accepted Student Day Event Attended  12043 non-null  int64  
 9   Campus Visits - Person               12043 non-null  int64  
 10  Campus Visits - App                  12043 non-null  int64  
 11  Logins Before Admit              

In [65]:
#fill the missing 5 values of Application ACRK using mean or median imputation
adm_data[adm_data['Application ACRK'].isnull()]

,Application Major,Application College,Admit Open Span,Scholarship_Awarded,Application CGPA,FAFSA Filed,High School Code,Financial Aid Appeal,Accepted Student Day Event Attended,Campus Visits - Person,...,Person Sex_M,Person Sex_Unknown,High School Region_Midwest,High School Region_Northeast,High School Region_South,High School Region_Southwest,High School Region_Territory,High School Region_West,Application Span,Admission Span
1472,27,CAS,NaN,0,0.0,0,70255.0,0,1,0,...,0,0,0,1,0,0,0,0,0,0.0
1473,85,TCOE,NaN,0,0.0,0,70255.0,0,0,0,...,0,0,0,1,0,0,0,0,406,0.0
8489,34,HCLC,NaN,1,0.0,0,70440.0,0,0,0,...,1,0,0,1,0,0,0,0,180,0.0
9485,53,TCOE,NaN,0,0.0,0,70040.0,0,0,0,...,1,0,0,1,0,0,0,0,811,0.0


In [66]:
#just drop the columns where admitted is 0 and acrk score is null
rows_to_drop = adm_data[(adm_data['admitted'] == 0) & (adm_data['Application ACRK'].isnull())]
adm_data = adm_data.drop(rows_to_drop.index).reset_index(drop=True)



In [67]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12040 entries, 0 to 12039
Data columns (total 42 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Application Major                    12040 non-null  int64  
 1   Application College                  12040 non-null  object 
 2   Admit Open Span                      6745 non-null   float64
 3   Scholarship_Awarded                  12040 non-null  int64  
 4   Application CGPA                     12040 non-null  float64
 5   FAFSA Filed                          12040 non-null  int64  
 6   High School Code                     12023 non-null  float64
 7   Financial Aid Appeal                 12040 non-null  int64  
 8   Accepted Student Day Event Attended  12040 non-null  int64  
 9   Campus Visits - Person               12040 non-null  int64  
 10  Campus Visits - App                  12040 non-null  int64  
 11  Logins Before Admit         

In [68]:
row_to_drop=adm_data[adm_data['Application Span']==-4]
adm_data=adm_data.drop(row_to_drop.index).reset_index(drop=True)

In [69]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12040 entries, 0 to 12039
Data columns (total 42 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Application Major                    12040 non-null  int64  
 1   Application College                  12040 non-null  object 
 2   Admit Open Span                      6745 non-null   float64
 3   Scholarship_Awarded                  12040 non-null  int64  
 4   Application CGPA                     12040 non-null  float64
 5   FAFSA Filed                          12040 non-null  int64  
 6   High School Code                     12023 non-null  float64
 7   Financial Aid Appeal                 12040 non-null  int64  
 8   Accepted Student Day Event Attended  12040 non-null  int64  
 9   Campus Visits - Person               12040 non-null  int64  
 10  Campus Visits - App                  12040 non-null  int64  
 11  Logins Before Admit         

In [70]:
adm_data['admitted'].value_counts()

admitted
0    11746
1      294
Name: count, dtype: int64

In [71]:
adm_data['Application ACRK'] = adm_data['Application ACRK'].fillna(adm_data['Application ACRK'].mean())

In [72]:
# Get the maximum value
max_value = adm_data['Admission Span'].max()  # Should be 324.467789
# Retrieve the row with that maximum value
max_row = adm_data[adm_data['Admission Span'] == max_value]
adm_data=adm_data.drop(max_row.index[0]).reset_index(drop=True)

In [73]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12039 entries, 0 to 12038
Data columns (total 42 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Application Major                    12039 non-null  int64  
 1   Application College                  12039 non-null  object 
 2   Admit Open Span                      6744 non-null   float64
 3   Scholarship_Awarded                  12039 non-null  int64  
 4   Application CGPA                     12039 non-null  float64
 5   FAFSA Filed                          12039 non-null  int64  
 6   High School Code                     12022 non-null  float64
 7   Financial Aid Appeal                 12039 non-null  int64  
 8   Accepted Student Day Event Attended  12039 non-null  int64  
 9   Campus Visits - Person               12039 non-null  int64  
 10  Campus Visits - App                  12039 non-null  int64  
 11  Logins Before Admit         

In [74]:
adm_data.to_excel('pre_processed_data.xlsx', index=False)

In [75]:
adm_data['admitted'].value_counts()

admitted
0    11746
1      293
Name: count, dtype: int64

In [76]:
#ACTUAL COUNT=1465

In [77]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12039 entries, 0 to 12038
Data columns (total 42 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Application Major                    12039 non-null  int64  
 1   Application College                  12039 non-null  object 
 2   Admit Open Span                      6744 non-null   float64
 3   Scholarship_Awarded                  12039 non-null  int64  
 4   Application CGPA                     12039 non-null  float64
 5   FAFSA Filed                          12039 non-null  int64  
 6   High School Code                     12022 non-null  float64
 7   Financial Aid Appeal                 12039 non-null  int64  
 8   Accepted Student Day Event Attended  12039 non-null  int64  
 9   Campus Visits - Person               12039 non-null  int64  
 10  Campus Visits - App                  12039 non-null  int64  
 11  Logins Before Admit         

In [78]:
adm_data['admitted'].value_counts()

admitted
0    11746
1      293
Name: count, dtype: int64

In [79]:
adm_data.drop(['Admit Open Span'],axis=1,inplace=True)

In [80]:
#handle null values for High School Code and remove them using highschool name  
#adm_data[adm_data['High School Code'].isna() & (adm_data['admitted']==0)]
adm_data = adm_data.drop(adm_data[adm_data['High School Code'].isna() & (adm_data['admitted'] == 0)].index).reset_index(drop=True)

In [81]:
adm_data['High School Code'].fillna('070905', inplace=True)


/var/folders/7t/938pkq393dl5_v0w4yxztbpw0000gn/T/ipykernel_2093/438156833.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  adm_data['High School Code'].fillna('070905', inplace=True)


In [82]:
adm_data['High School Code'].value_counts()

High School Code
70910.0     100
70055.0     100
70120.0      74
311120.0     69
70665.0      66
           ... 
331145.0      1
331920.0      1
341675.0      1
53470.0       1
311025.0      1
Name: count, Length: 2991, dtype: int64

In [83]:
# Ensure the column is in string format if it isn't already
adm_data['High School Code'] = adm_data['High School Code'].astype(str)

# Initialize the label encoder
encoder = LabelEncoder()

# Fit and transform the 'High School Code' column
adm_data['High School Code'] = encoder.fit_transform(adm_data['High School Code'])


In [84]:
#2 convert application to two categories using one hot encoding or label encoder
encoder= OneHotEncoder(sparse_output=False, dtype=int)
encoded_array = encoder.fit_transform(adm_data[['Application College']])
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['Application College']))
adm_data = pd.concat([adm_data, encoded_df], axis=1)

In [85]:
adm_data.drop(columns=['Application College'], inplace=True)
#drop the column else we get a new dataframe with the columns 

In [86]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12022 entries, 0 to 12021
Data columns (total 46 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Application Major                    12022 non-null  int64  
 1   Scholarship_Awarded                  12022 non-null  int64  
 2   Application CGPA                     12022 non-null  float64
 3   FAFSA Filed                          12022 non-null  int64  
 4   High School Code                     12022 non-null  int64  
 5   Financial Aid Appeal                 12022 non-null  int64  
 6   Accepted Student Day Event Attended  12022 non-null  int64  
 7   Campus Visits - Person               12022 non-null  int64  
 8   Campus Visits - App                  12022 non-null  int64  
 9   Logins Before Admit                  12022 non-null  int64  
 10  Acceptance Call Success              12022 non-null  int64  
 11  Application Consider Test Sc

In [87]:
# Adding new columns with all zeros
adm_data['Address 1 Region_Military'] = 0
adm_data['High School Region_Military'] = 0


In [88]:
adm_data['Person Sex_Unknown']=0

In [89]:
adm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12022 entries, 0 to 12021
Data columns (total 48 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Application Major                    12022 non-null  int64  
 1   Scholarship_Awarded                  12022 non-null  int64  
 2   Application CGPA                     12022 non-null  float64
 3   FAFSA Filed                          12022 non-null  int64  
 4   High School Code                     12022 non-null  int64  
 5   Financial Aid Appeal                 12022 non-null  int64  
 6   Accepted Student Day Event Attended  12022 non-null  int64  
 7   Campus Visits - Person               12022 non-null  int64  
 8   Campus Visits - App                  12022 non-null  int64  
 9   Logins Before Admit                  12022 non-null  int64  
 10  Acceptance Call Success              12022 non-null  int64  
 11  Application Consider Test Sc

In [90]:
adm_data.to_excel('fall3325.xlsx', index=False)